# Num PCA components
Use explained variance and reconstruction error to determine the number of PCA components to use.

In [11]:
from elasticSearch.selected_docs import select_rep_path
import os
from elasticSearch.recursive_search import scanRecurse

## Data
- local: 195 (first) files
- server: 1000 random files across subdirectories

In [12]:
def get_directories():
    baseDir = '/mnt/stud/home/kgutekunst/visualizations/images/'
    resDir = '/mnt/stud/home/kgutekunst/visualizations/'
    if os.path.exists('/Users/klara/Documents/uni/bachelorarbeit/data/0/'):
        baseDir = '/Users/klara/Documents/uni/bachelorarbeit/images/'
        resDir = '/Users/klara/Developer/Uni/topic-analysis-text-data/results/'
    return baseDir,resDir

In [13]:
baseDir, resDir = get_directories()

In [14]:
paths = select_rep_path(baseDir, 1000) if baseDir.startswith('/mnt/') else list(scanRecurse(baseDir))

In [15]:
import numpy as np


def rgb2gray(img):  # not normalized to [0,1]
    return 0.299*img[:,:,0] + 0.587*img[:,:,1] + 0.114*img[:,:,2]

def get_maximum_height_width(documents):
    max_w = 0
    max_h = 0
    for doc in documents:
        max_w = np.maximum(max_w, doc.shape[0])
        max_h = np.maximum(max_h, doc.shape[1])
    return max_w, max_h

def proprocess_docs(raw_documents, max_w, max_h):
    '''
    return same sized, greyscale documents as an array
    '''
    documents = []
    for doc in raw_documents:
        # same size for all documents
        C = np.ones((max_w,max_h))
        # convert to grayscale
        C[:doc.shape[0],:doc.shape[1]] = rgb2gray(doc)
        # 2d to 1d array
        documents.append(C.ravel())
    # list to array
    return np.asarray(documents)

In [16]:
from matplotlib import pyplot as plt


def plot_expl_var(pca, save=False, resDir:str='results/'):
    fig, ax = plt.subplots(figsize=(10,6))
    y = np.cumsum(pca.explained_variance_ratio_)
    plt.plot(y, label="cumulative explained variance")
    plt.title("Cumulative explained variance")
    plt.xlabel("Number of components")
    plt.ylabel("Explained variance")
    plt.axhline(y = 0.9, color = 'r', linestyle = '-', label="90% explained variance")
    interp = np.interp(0.9, y, list(range(len(y))))
    plt.axvline(x=interp,color='grey', label="{} components".format(int(interp)))
    plt.legend()
    if save:
        plt.savefig(resDir + "cumulative_explained_variance.pdf", format="pdf")
    plt.show()

In [17]:
from sklearn.decomposition import PCA


def plot_rec_err(X_train, X_test, n_max=10, save=False, resDir:str='results/'):
    reconstr_err = []
    for i in range(1, n_max):
        pca = PCA(n_components=i, whiten=True, svd_solver="randomized")   
        pca.fit(X_train)
        X_test_pca = pca.transform(X_test)
        X_test_pca_inverse = pca.inverse_transform(X_test_pca)
        reconstr_err.append(np.linalg.norm(X_test_pca_inverse - X_test) / np.sqrt(X_test.shape[0]))

    # plot reconstruction error
    plt.figure(figsize=(10,6))
    plt.plot(reconstr_err, label="reconstruction error")
    plt.title("Reconstruction error")
    plt.xlabel("Number of components")
    plt.ylabel("Reconstruction error")
    plt.legend()
    if save:
        plt.savefig(resDir + "reconstruction error.pdf", format="pdf")
    plt.show()
    

In [18]:
documents_raw = [plt.imread(fp) for fp in paths if fp.endswith(".png")]

In [19]:
max_w, max_h = get_maximum_height_width(documents_raw)
print(max_w, max_h)

880 1026


In [20]:
eigendocs = proprocess_docs(documents_raw, max_w=max_w, max_h=max_h)

In [21]:
from sklearn.model_selection import train_test_split


X_train, X_test = train_test_split(eigendocs, random_state=0)

In [22]:
pca = PCA(n_components=min(len(X_train), len(X_train[0])), whiten=True, svd_solver="randomized")   
pca.fit(X_train)

In [ ]:
plot_expl_var(pca, save=True, resDir=resDir)

In [ ]:
plot_rec_err(X_train, X_test, n_max= min(len(X_train),len(X_train[0])), save=True, resDir=resDir)